In [1]:
import pandas as pd
import glob
import os
pd.options.mode.chained_assignment = None
source_dir = "download"
start_dir = os.getcwd()
dest_dir = "data"
#06_aggregate_data_osm_igm_comparison.py

In [2]:
# search all files inside a specific folder
# *.* means file name with any extension
os.chdir(source_dir)
dir_path = r'*.parquet'
data_files = glob.glob(dir_path)
datafiles = []
for f in data_files:
    if (f.find("_dbsn") == -1):
        datafiles.append(pd.read_parquet(f))
data_igm_osm = pd.concat(datafiles)
data_igm_osm = data_igm_osm.fillna(0).reset_index()
del data_igm_osm['index']
columns = data_igm_osm.columns.unique()

In [5]:
data_igm_osm

,area_osm_buildings,area_igm_buildings,area_osm_in_igm_buildings,osm_in_igm_streets_length,osm_streets_length,igm_streets_length,province,region
0,3.227615e+07,6.132947e+07,3.207791e+06,6.462345e+06,6.745383e+06,1.826025e+07,Perugia,Umbria
1,7.085357e+06,1.449599e+07,3.394135e+05,1.765620e+06,1.843504e+06,4.251947e+06,Crotone,Calabria
2,1.557832e+08,1.786213e+08,6.035078e+07,3.348433e+07,3.497882e+07,4.603448e+07,Roma,Lazio
3,2.365442e+07,2.644279e+07,1.611575e+07,7.004669e+06,7.273045e+06,9.251229e+06,Arezzo,Toscana
4,7.707583e+06,1.838024e+07,2.999176e+05,1.952329e+06,2.009184e+06,6.239436e+06,Rieti,Lazio
5,8.607381e+07,7.861639e+07,6.513626e+07,2.270665e+07,2.451109e+07,2.206313e+07,Lecce,Puglia
6,4.020889e+07,3.798529e+07,3.184032e+07,9.641589e+06,1.045153e+07,1.002823e+07,Foggia,Puglia
7,3.362154e+07,3.207722e+07,2.632024e+07,9.557488e+06,1.011436e+07,9.325345e+06,Brindisi,Puglia
8,1.656218e+07,1.714460e+07,1.120964e+07,5.011388e+06,5.329202e+06,6.293609e+06,Grosseto,Toscana
9,4.083966e+07,2.443017e+07,1.578427e+07,5.560660e+06,1.015378e+07,6.269354e+06,Cagliari,Sardegna


In [10]:
# conversion in km
data_igm_osm.area_igm_buildings = data_igm_osm.area_igm_buildings / 1000
data_igm_osm.osm_in_igm_streets_length = data_igm_osm.osm_in_igm_streets_length / 1000
data_igm_osm.area_igm_buildings = data_igm_osm.area_igm_buildings / 1000
data_igm_osm.area_osm_buildings = data_igm_osm.area_osm_buildings / 1000
data_igm_osm.osm_streets_length = data_igm_osm.osm_streets_length / 1000
data_igm_osm.igm_streets_length = data_igm_osm.igm_streets_length / 1000
data_igm_osm.area_osm_in_igm_buildings = data_igm_osm.area_osm_in_igm_buildings / 1000
data_igm_osm.osm_in_igm_streets_length = data_igm_osm.osm_in_igm_streets_length / 1000


In [4]:
df1 = data_igm_osm.groupby("region")['area_igm_buildings'].sum().to_frame().reset_index()
df2 = data_igm_osm.groupby("region")['area_osm_buildings'].sum().to_frame().reset_index()
df3 = data_igm_osm.groupby("region")['area_osm_in_igm_buildings'].sum().to_frame().reset_index()

In [5]:
df4 = data_igm_osm.groupby("region")['igm_streets_length'].sum().to_frame().reset_index()
df5 = data_igm_osm.groupby("region")['osm_streets_length'].sum().to_frame().reset_index()
df6 = data_igm_osm.groupby("region")['osm_in_igm_streets_length'].sum().to_frame().reset_index()


In [6]:
regions = df1.merge(df2).merge(df3).merge(df4).merge(df5).merge(df6)

In [7]:
regions.to_csv(start_dir + os.sep + dest_dir + os.sep + "regions_osm_igm_comparison.csv",index=False)

In [8]:
data_igm_osm.to_csv(start_dir + os.sep + dest_dir + os.sep + "provinces_osm_igm_comparison.csv",index=False)

In [22]:
regions['percentage_area_osm_in_igm_buildings'] = round((regions.area_osm_in_igm_buildings / regions.area_igm_buildings) * 100,2)

In [23]:
regions

,region,area_igm_buildings,area_osm_buildings,area_osm_in_igm_buildings,igm_streets_length,osm_streets_length,osm_in_igm_streets_length,percentage_area_osm_in_igm_buildings
0,Abruzzo,1.178672e+08,4.439872e+07,2.624932e+06,3.474718e+07,9.577013e+06,9.038182e+06,2.23
1,Basilicata,5.236806e+07,2.066701e+07,3.580216e+06,1.579580e+07,5.780160e+06,5.575766e+06,6.84
2,Calabria,1.609769e+08,4.354385e+07,1.196286e+06,5.193654e+07,1.088651e+07,1.035931e+07,0.74
3,Campania,3.121999e+08,1.448794e+08,7.087781e+07,8.155792e+07,3.229715e+07,3.049933e+07,22.70
4,Lazio,3.363827e+08,2.195504e+08,5.613148e+07,9.368037e+07,4.731921e+07,4.490254e+07,16.69
5,Marche,1.116983e+08,4.854240e+07,1.461339e+07,2.722365e+07,1.014334e+07,9.085278e+06,13.08
6,Molise,2.677820e+07,1.226751e+07,4.385139e+06,9.392340e+06,2.931250e+06,2.675969e+06,16.38
7,Puglia,2.728499e+08,2.902039e+08,2.239555e+08,7.306010e+07,7.886108e+07,7.336181e+07,82.08
8,Sardegna,1.438796e+08,1.359194e+08,7.553971e+07,4.121963e+07,3.599135e+07,3.033493e+07,52.50
9,Sicilia,3.312632e+08,1.416843e+08,2.479455e+07,9.129750e+07,3.694064e+07,3.441953e+07,7.48


In [13]:
regions[regions.area_igm_buildings <= regions.area_osm_buildings]

,region,area_igm_buildings,area_osm_buildings,area_osm_in_igm_buildings,igm_streets_length,osm_streets_length,osm_in_igm_streets_length
7,Puglia,2.728499e+08,2.902039e+08,2.239555e+08,7.306010e+07,7.886108e+07,7.336181e+07


In [14]:
regions[regions.igm_streets_length <= regions.osm_streets_length]

,region,area_igm_buildings,area_osm_buildings,area_osm_in_igm_buildings,igm_streets_length,osm_streets_length,osm_in_igm_streets_length
7,Puglia,2.728499e+08,2.902039e+08,2.239555e+08,7.306010e+07,7.886108e+07,7.336181e+07
